In [0]:
%tensorflow_version 2.x
import tensorflow as tf


In [0]:
#@title 1
@tf.function
def top_k(input_tensor, k):
  return tf.math.top_k(input_tensor, k).values

a = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
k = 1
print(top_k(a,k))


In [0]:
#@title 2
def arg_max_tensor(a):
  b = tf.math.top_k(a,k=1).indices

  c = tf.TensorArray(tf.int32, size= 0, dynamic_size= True)
  for i in tf.range(tf.shape(a)[0]): 

    indices = b[i]
    indices = tf.expand_dims(indices,0) 
    print(indices)
    updates = tf.Variable([1])  
    shape = tf.Variable([tf.shape(a)[1]]) 

    scatter = tf.scatter_nd(indices, updates, shape)
    c.write(i,scatter).mark_used()

  return c.stack()

a = tf.Variable([[3,2,3],[4,6,5],[7,8,9],[11,12,13]])
m = arg_max_tensor(a)
print(m)


In [0]:
#@title 3
def arg_k_max_tensor(a,k):
  
  b = tf.math.top_k(a,k=k).indices
  c = tf.TensorArray(tf.int32, size= 0, dynamic_size= True)

  for i in tf.range(tf.shape(a)[0]): 

    indices = b[i]
    indices = tf.expand_dims(indices,0)
    
    d = tf.TensorArray(tf.int32, size= 0, dynamic_size= True)
    for j in tf.range(tf.shape(indices)[1]):
      d.write(j, [indices[0][j]]).mark_used()
    d = d.stack()
    
    updates = tf.Variable(tf.ones(dtype= tf.int32,shape = tf.shape(d)[0]))    
    shape = tf.Variable([tf.shape(a)[1]])  
    scatter = tf.scatter_nd(d, updates, shape)
    c.write(i,scatter).mark_used()

  return c.stack()


a = tf.Variable([[1000,1,0,2,3],[4,5,6,2,1],[7,8,9,88,99],[10,11,12,13,14]])
k = 3
print(arg_k_max_tensor(a,k))






In [0]:
#@title 4

def exp_moving(a, decay_rate):
  b = tf.TensorArray(dtype = tf.float32, size= 0, dynamic_size= True)
  decay_rate = tf.Variable(decay_rate, dtype= tf.float32)
  

  for i in tf.range(0,tf.shape(a), delta=1):
    if i == 0:
      b.write(i,a[i]).mark_used()
    else:
      m = tf.math.multiply(decay_rate,b.stack()[i-1])
      
      n = tf.math.multiply((1-decay_rate),a[i])
      
      c = m + n 
      b.write(i,c).mark_used()
  return b.stack()

a = tf.Variable([100,200, 300], dtype= tf.float32)
#print(a)
decay_rate = 0.33
tf.print(exp_moving(a, decay_rate))



In [0]:
#@title 4 with tf function
b = tf.TensorArray(dtype = tf.float32, size= 0, dynamic_size= True)
@tf.function
def exp_moving(a, decay_rate):
  
  decay_rate = tf.Variable(decay_rate, dtype= tf.float32)
  sh = tf.convert_to_tensor(tf.shape(a)[0])
  #print(sh)

  for i in tf.range(0,sh, delta=1):
    
   
    def f1(): b.write(i,a[i]).mark_used()
    
    
    def f2(): 
      m = tf.math.multiply(decay_rate,b.stack()[i-1])      
      n = tf.math.multiply((1-decay_rate),a[i])      
      c = m + n 
      b.write(i,c).mark_used()
   
    tf.cond(i==0, f1,f2)
   
    
    # if i == 0:
    #   b.write(i,a[i]).mark_used()
    # else:
    #   m = tf.math.multiply(decay_rate,b.stack()[i-1])
      
    #   n = tf.math.multiply((1-decay_rate),a[i])
      
    #   c = m + n 
    #   b.write(i,c).mark_used()
  return b.stack()

a = tf.Variable([100,200, 300], dtype= tf.float32)
#print(a)
decay_rate = 0.33
tf.print(exp_moving(a, decay_rate))

In [0]:
#@title 5
a = tf.Variable([100,200,300], dtype= tf.float32)
rate = 0.5

decay_rate = tf.Variable(rate, dtype= tf.float32)
n = tf.cast(tf.shape(a)[0], dtype= tf.float32)
print(n)
b = tf.math.multiply(tf.math.pow(decay_rate,n),a [0])
print(b)
r = tf.range(1,n)

# def sum(decay_rate, n):
#   return tf.math.reduce_sum(tf.map_fn(lambda i :tf.math.multiply(tf.math.multiply(tf.math.pow(decay_rate,n-i),1-decay_rate),i) ,(a, tf.range(1,n))))
# c = sum(decay_rate, n)
# print(c)



In [0]:
mysum(t, N):
    return sum(map(lambda n : sin(t*n*n), range(N))

In [0]:
#@title 6
def odd_eve(x,y,z):

  m = tf.TensorArray(dtype= tf.float32, size= 0, dynamic_size= True)
  if tf.shape(x).shape == 1:
    for i in tf.range(tf.shape(x)[0]):
      if (x[i]%2 == 0):
        m.write(i,y[i]).mark_used()
      else:
        m.write(i, z[i]).mark_used()
  else:

    for i in tf.range(tf.shape(x)[0]):
      n = tf.TensorArray(dtype= tf.float32, size= 0, dynamic_size= True)
      for j in tf.range(tf.shape(x)[1]):
        if (x[i][j]% 2 == 0):
          n.write(j,y[i][j]).mark_used()
        else:
          n.write(j,z[i][j]).mark_used()
      n = n.stack()
      m.write(i,n).mark_used()

  return m.stack()

x = tf.Variable([[1,2,3],[4,5,6],[7,8,9]], dtype= tf.float32)
y = tf.Variable([[10,11,12],[13,14,15],[16,17,18]], dtype= tf.float32)
z = tf.Variable([[19,20,21],[22,23,24],[25,26,27]], dtype= tf.float32)

print(odd_eve(x,y,z))





In [0]:
#@title 7
def last_dim(x):
  if tf.shape(x)[-1] > 100:
     return 100
  elif tf.shape(x)[-1] >44:
     return 12
  else:
    return 0
 
x = tf.Variable(tf.zeros(shape = (100,44,55,66)))
print(last_dim(x))


In [0]:
#@title 8
count_100 = 0
count_44 = 0
count_0 = 0

In [0]:
#@title 8

def last_dim(x):
  if tf.shape(x)[-1] > 100:
    global count_100
    count_100 = count_100 + 1
    return 100
  elif tf.shape(x)[-1] >44:
    global count_44
    count_44 = count_44 + 1
    return 12
  else:
    global count_0
    count_0 = count_0 + 1
    return 0


 
x = tf.Variable(tf.zeros(shape = (100,44,55,101)))
print(last_dim(x))
print("Count100 :", count_100)
print("Count44 :", count_44)
print("Count0 :", count_0)


In [0]:
#@title 9
@tf.function
def broad_cast(a,b):
  return a- tf.reshape(b,[tf.shape(b)[0],1])


a = tf.Variable([1,2,3], dtype = tf.float32)
b = tf.Variable([4,5,6], dtype = tf.float32)

print(broad_cast(a,b))


In [0]:
#@title 10
encoder_states = tf.Variable([[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],[[5,5,5,5],[6,6,6,6],[7,7,7,7],[8,8,8,8]],[[9,9,9,9],[10,10,10,10],[11,11,11,11],[12,12,12,12]]] ,dtype= tf.float32)
#print(tf.shape(encoder_states))
decoder_state = tf.Variable(tf.ones((4,4)), dtype= tf.float32)
#print(tf.shape(decoder_state))
alpha = tf.tensordot(encoder_states, decoder_state, axes= [[-1],[-1]])
alpha = tf.reduce_mean(alpha, axis=2)
#print(tf.shape(alpha))
print(alpha)